## explore data

In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print("length of dataset in characters: ", len(text))
print(text[:1000])

In [3]:
chars= sorted(list(set(text)))
vocab_size = len(chars)
print("Vocabulary size: ", vocab_size)
print(chars)
char_to_idx = {ch:i for i,ch in enumerate(chars)}
idx_to_char = {i:ch for i,ch in enumerate(chars)}

encode = lambda x: [char_to_idx[ch] for ch in x]
decode = lambda x: ''.join([idx_to_char[i] for i in x])
print("Encoded: ", encode("hello"))
print("Decoded: ", decode(encode("hello")))

Vocabulary size:  65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Encoded:  [46, 43, 50, 50, 53]
Decoded:  hello


In [4]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:10])

/home/saidinesh/Desktop/Projects/nano_gpt_demo/.venv/lib/python3.10/site-packages/torch/_subclasses/functional_tensor.py:258: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [5]:
# split the data into train and test
n= data.shape[0]
train_data = data[:int(0.8*n)]
val_data = data[int(0.8*n):]

In [6]:
block_size =8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [7]:
x= train_data[:block_size]
y= train_data[1:block_size+1]
for t in range(block_size):
    context =x[:t+1]
    target = y[t]
    print(f'when we see {context} we predict {target}')

when we see tensor([18]) we predict 47
when we see tensor([18, 47]) we predict 56
when we see tensor([18, 47, 56]) we predict 57
when we see tensor([18, 47, 56, 57]) we predict 58
when we see tensor([18, 47, 56, 57, 58]) we predict 1
when we see tensor([18, 47, 56, 57, 58,  1]) we predict 15
when we see tensor([18, 47, 56, 57, 58,  1, 15]) we predict 47
when we see tensor([18, 47, 56, 57, 58,  1, 15, 47]) we predict 58


BatchDimension

In [9]:
torch.manual_seed(0)
block_size= 8# sequence length
batch_size = 4# number of sequences in a batch

def get_batch(split):
    # generate sequences of length block_size
    data= train_data if split=='train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x= torch.stack([data[i:i+block_size] for i in ix])
    y= torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb,yb= get_batch('train')
print(xb.shape, yb.shape)
print(xb)
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when we see {context} we predict {target}')

torch.Size([4, 8]) torch.Size([4, 8])
tensor([[39, 60, 43,  1, 21,  1, 58, 46],
        [ 1, 63, 53, 59,  6,  1, 57, 47],
        [ 1, 54, 56, 53, 59, 42,  1, 46],
        [ 1, 58, 46, 53, 59,  1, 44, 39]])
tensor([[60, 43,  1, 21,  1, 58, 46, 53],
        [63, 53, 59,  6,  1, 57, 47, 56],
        [54, 56, 53, 59, 42,  1, 46, 43],
        [58, 46, 53, 59,  1, 44, 39, 47]])
when we see tensor([39]) we predict 60
when we see tensor([39, 60]) we predict 43
when we see tensor([39, 60, 43]) we predict 1
when we see tensor([39, 60, 43,  1]) we predict 21
when we see tensor([39, 60, 43,  1, 21]) we predict 1
when we see tensor([39, 60, 43,  1, 21,  1]) we predict 58
when we see tensor([39, 60, 43,  1, 21,  1, 58]) we predict 46
when we see tensor([39, 60, 43,  1, 21,  1, 58, 46]) we predict 53
when we see tensor([1]) we predict 63
when we see tensor([ 1, 63]) we predict 53
when we see tensor([ 1, 63, 53]) we predict 59
when we see tensor([ 1, 63, 53, 59]) we predict 6
when we see tensor([ 1, 

## Bigram Model (testing the data with simplest Model)

In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers and C number of classes(characters)
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

    
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.9475, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [11]:
optimizer = torch.optim.Adam(m.parameters(), lr=0.01)
batch_size = 32

for i in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if i % 1000 == 0:
        print(f'iteration {i}, loss = {loss.item()}')

iteration 0, loss = 4.664743900299072
iteration 1000, loss = 2.4751906394958496
iteration 2000, loss = 2.5420734882354736
iteration 3000, loss = 2.4348995685577393
iteration 4000, loss = 2.418869972229004
iteration 5000, loss = 2.489229679107666
iteration 6000, loss = 2.4532127380371094
iteration 7000, loss = 2.4838154315948486
iteration 8000, loss = 2.505552053451538
iteration 9000, loss = 2.4648795127868652


In [12]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


IZENOng h hasbe pave pirance
RDe hicomyonthar's
PES:
AK:
Mith henouratucenonthioneir thondy, y helti


## Self Atention Demo

In [13]:
B,T,C = 4,8,2
logits = torch.randn(B,T,C)
logits.shape

torch.Size([4, 8, 2])

In [14]:
# we are doing logits[b,t]= mean_{i<=t} logit[b,i]
logitsbow = torch.zeros(B,T,C)
for b in range(B):
    for t in range(T):
        logitsprev = logits[b,:t+1] # t,C
        logitsbow[b,t] = logitsprev.mean(dim=0)
        
logits[0]


tensor([[-3.5685e-01, -1.4841e+00],
        [ 1.4718e-03, -9.1119e-01],
        [-4.2776e-01, -1.4516e+00],
        [ 5.2601e-01, -2.5594e-02],
        [ 4.3046e-01, -9.9963e-01],
        [-6.9346e-01,  9.3705e-01],
        [-1.1372e+00, -1.1862e+00],
        [-9.0355e-01, -1.1036e+00]])

In [15]:
logitsbow[0]

tensor([[-0.3568, -1.4841],
        [-0.1777, -1.1976],
        [-0.2610, -1.2823],
        [-0.0643, -0.9681],
        [ 0.0347, -0.9744],
        [-0.0867, -0.6558],
        [-0.2368, -0.7316],
        [-0.3201, -0.7781]])

In [16]:
# optimizing abovve code with matrix operations
torch.manual_seed(0)
wei= torch.tril(torch.ones(T,T))
wei= wei/wei.sum(dim=1, keepdim=True)
print(wei)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


In [17]:
logitsbow2= wei@ logits # (B,T,T) @ (B,T,C) -> (B,T,C)
torch.allclose(logitsbow, logitsbow2)
print(logitsbow2[0], logitsbow[0])

tensor([[-0.3568, -1.4841],
        [-0.1777, -1.1976],
        [-0.2610, -1.2823],
        [-0.0643, -0.9681],
        [ 0.0347, -0.9744],
        [-0.0867, -0.6558],
        [-0.2368, -0.7316],
        [-0.3201, -0.7781]]) tensor([[-0.3568, -1.4841],
        [-0.1777, -1.1976],
        [-0.2610, -1.2823],
        [-0.0643, -0.9681],
        [ 0.0347, -0.9744],
        [-0.0867, -0.6558],
        [-0.2368, -0.7316],
        [-0.3201, -0.7781]])


In [20]:
torch.manual_seed(0)
B,T,C= 4,8,2
x= torch.randn(B,T,C)

tril= torch.tril(torch.ones(T,T))
wei = tril/tril.sum(dim=1, keepdim=True)
wei = wei.masked_fill(wei==0, float('-inf'))
wei= F.softmax(wei, dim=1)
out = wei@x
print(out[0])

tensor([[-1.1258, -1.1524],
        [-0.6882, -0.7931],
        [-0.1759, -0.2981],
        [-0.2109, -0.7524],
        [-0.1043, -0.8546],
        [-0.0286, -0.6608],
        [-0.0074, -0.3896],
        [ 0.1331, -0.3718]])


In [21]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

v4 self attenstion

In [23]:
torch.manual_seed(0)
B,T,C= 4,8,2
x= torch.randn(B,T,C)

head_size  =16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B,T, head_size)
q = query(x) # (B,T, head_size)

wei = q@k.transpose(-2,-1) # (B,T, head_size) @ (B, head_size, T) -> (B,T,T)


tril= torch.tril(torch.ones(T,T))
wei = tril/tril.sum(dim=1, keepdim=True)
wei = wei.masked_fill(wei==0, float('-inf'))
wei= F.softmax(wei, dim=1)
v= value(x) # (B,T, head_size)
out = wei@v # (B,T,T) @ (B,T, head_size) -> (B,T, head_size)
print(out.shape)

torch.Size([4, 8, 16])


Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below